# Project 3 - Group 1

Write a script to analyze the flight data from PS2.
- The following are required elements of the final script(s) that are turned in:
    - You must have a database with at least 2 tables
    - Each table should have a primary key 
    - There should be at least one foreign key
    - You can either set up the tables in sqlite or from your Python script.
- Your Python script should use SQLAlchemy to
    - Add data to the database
    - Query data in the database using each of these commands:
        - select, where, join
    - Preferably uses the SQL Expression Language, but OK if you use raw SQL.
    
# Data used is flights.1k.csv and it is stored in subfolder "data/.."

# Create database with 2 tables

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import DateTime, Boolean
from sqlalchemy import exists
from sqlalchemy import sql, select, join, desc


# Create a sqlite database 
engine = create_engine('sqlite:///data/flights.project3.sqlite')

metadata = MetaData(engine)

# Try to load Airports info from database; if not there, create it.
try:
    Airports = Table('Airports', metadata, autoload=True)
except:
    Airports = Table ('Airports', metadata,
                Column('ID', Integer, autoincrement=True),
                Column('Code', String, primary_key=True),
                Column('City', String),
                Column('State', String),
                Column('Name', String),
               )

# Same for Flights table.
try:
    Flights = Table('Flights', metadata, autoload=True)
except:
    Flights = Table ('Flights', metadata,
                 Column('ID', Integer, autoincrement=True, primary_key=True),
                 Column('Fl_date', DateTime),
                 Column('Airline_ID', String),
                 Column('Origin', String, ForeignKey("Airports.Code")),
                 Column('Destination', String, ForeignKey("Airports.Code")),
                 Column('Dep_Time', String),
                 Column('Dep_Delay_New', Integer),
                 Column('Arr_Time', String),
                 Column('Arr_Delay_New', Integer),
                 Column('Cancelled', Boolean),
                 Column('Cancellation_Code', String),
                 Column('Diverted', Boolean),
                 Column('Air_Time', Integer),
                 Column('Flights', Integer),
                 Column('Distance', Integer),
                 Column('Carrier_Delay', Integer),
                 Column('Weather_Delay', Integer),
                 Column('NAS_Delay', Integer),
                 Column('Security_Delay', Integer),
                 Column('Late_Aircraft_Delay', Integer)
                )
                 
                 
metadata.create_all(engine)

# Add data to sqlite database

In [2]:
import csv

flights = open("data/flights.1K.csv")


reader = csv.DictReader(flights)

Airport_dict = {}

# Read through the file and make a dictionary for airport codes.
# This gets a unique list of airport codes.
for Line in reader:
    if Line['ORIGIN'] not in Airport_dict:
        Airport_dict[Line['ORIGIN']]=[Line['ORIGIN_CITY_NAME'], Line['ORIGIN_STATE_ABR']]

    if Line['DEST'] not in Airport_dict:
        Airport_dict[Line['DEST']]=[Line['DEST_CITY_NAME'], Line['DEST_STATE_ABR']]


In [3]:
# Add the Airport_dict codes to the Airports table

conn = engine.connect()


def insert_airport(code,city,state):
    ins = Airports.insert().values(Code=code,
                                 City=city,
                                 State=state)
    
    result = conn.execute(ins)

for key, value in Airport_dict.items(): 
    insert_airport(key, value[0], value[1])

In [4]:
import datetime
import pandas as pd

def to_date(dates, lookup=False, **args):
    if lookup:
        return dates.map({v: pd.to_datetime(v, **args) for v in dates.unique()})
    return pd.to_datetime(dates, **args)

flights.close()
flights = open("data/flights.1K.csv")
reader = csv.DictReader(flights)
for Line in reader:
    try:
        Air_Time = float(Line['AIR_TIME'])
    except:
        Air_Time = 0
    ins=Flights.insert().values(Fl_date=to_date(Line['FL_DATE']),
                                Airline_ID = Line['AIRLINE_ID'],
                                Origin = Line['ORIGIN'],
                                Destination = Line['DEST'],
                                Dep_Time = Line['DEP_TIME'],
                                Dep_Delay_New = Line['DEP_DELAY_NEW'],
                                Arr_Time = Line['ARR_TIME'],
                                Arr_Delay_New = Line['ARR_DELAY_NEW'],
                                Cancelled = int(float((Line['CANCELLED']))),
                                Cancellation_Code = Line['CANCELLATION_CODE'],
                                Diverted = int(float((Line['DIVERTED']))),
                                Air_Time = Air_Time,
                                Flights = Line['FLIGHTS'],
                                Distance = Line['DISTANCE'],
                                Carrier_Delay = Line['CARRIER_DELAY'],
                                Weather_Delay = Line['WEATHER_DELAY'],
                                NAS_Delay = Line['NAS_DELAY'],
                                Security_Delay = Line['SECURITY_DELAY'],
                                Late_Aircraft_Delay = Line['LATE_AIRCRAFT_DELAY']
                                          )
    result = conn.execute(ins)
    

In [5]:
#from sqlalchemy import func
#query = select([func.max(Flights.c.Air_Time)])
#result = conn.execute(query)
#for row in result:
    #print(row)

# Select
Select all origin and destination pairs

In [6]:
from sqlalchemy import select

query = select([Flights.c.Origin, Flights.c.Destination])
             
result = conn.execute(query)
for row in result:
    print(row)

('LAX', 'IAD')
('LAX', 'IAD')
('SAN', 'SFO')
('SFO', 'EWR')
('JFK', 'SFO')
('LAX', 'OGG')
('OGG', 'LAX')
('SFO', 'JFK')
('LAX', 'EWR')
('EWR', 'LAX')
('LAX', 'EWR')
('EWR', 'LAX')
('LAX', 'EWR')
('EWR', 'LAX')
('EWR', 'LAX')
('LAX', 'EWR')
('SFO', 'SEA')
('SEA', 'SFO')
('SFO', 'EWR')
('SFO', 'EWR')
('SEA', 'LAX')
('SFO', 'EWR')
('EWR', 'SFO')
('EWR', 'SFO')
('SFO', 'LAX')
('LAX', 'SFO')
('EWR', 'SFO')
('SFO', 'LAX')
('EWR', 'SFO')
('SFO', 'DCA')
('SFO', 'ORD')
('ORD', 'SFO')
('SFO', 'AUS')
('AUS', 'SFO')
('SFO', 'PDX')
('AUS', 'SFO')
('SFO', 'AUS')
('SFO', 'JFK')
('ORD', 'LAX')
('LAX', 'ORD')
('LAX', 'ORD')
('JFK', 'SFO')
('ORD', 'LAX')
('SFO', 'JFK')
('JFK', 'LAS')
('LAS', 'SFO')
('JFK', 'SFO')
('LAS', 'JFK')
('SFO', 'JFK')
('JFK', 'SFO')
('JFK', 'SFO')
('DCA', 'SFO')
('LAX', 'MCO')
('MCO', 'LAX')
('FLL', 'LAX')
('LAX', 'FLL')
('FLL', 'LAX')
('SFO', 'FLL')
('FLL', 'SFO')
('SFO', 'JFK')
('SFO', 'BOS')
('BOS', 'SFO')
('SAN', 'SFO')
('SFO', 'BOS')
('BOS', 'SFO')
('BOS', 'SFO')
('SFO', 'B

('SEA', 'SFO')
('DEN', 'LAX')
('SEA', 'SFO')
('SFO', 'LAX')
('AUS', 'EWR')
('IAD', 'BOS')
('SMF', 'IAD')
('LAX', 'BOS')
('ORF', 'ORD')
('DEN', 'ORD')
('PDX', 'DEN')
('DEN', 'DTW')
('DSM', 'DEN')
('EWR', 'FLL')
('FLL', 'IAH')
('IAH', 'LAX')
('BOI', 'DEN')
('LAS', 'SFO')
('SJU', 'ORD')
('ORD', 'MSY')
('MSP', 'ORD')
('IAD', 'BDL')
('LAX', 'IAD')
('IAH', 'DFW')
('SFO', 'IAD')
('FLL', 'IAH')
('IAH', 'PHX')
('AUS', 'SFO')
('ORD', 'CMH')
('IAD', 'LAX')
('BNA', 'EWR')
('AUS', 'IAH')
('IAH', 'MSY')
('ALB', 'ORD')
('LAS', 'EWR')
('ORD', 'LAS')
('EWR', 'MSY')
('ORD', 'MSP')
('LAX', 'ORD')
('ORD', 'LGA')
('EWR', 'SFO')
('RSW', 'EWR')


# Where
Select where air time is greater than 100, then print origin and destination

In [7]:
query2 = select([Flights.c.Origin, Flights.c.Destination, Flights.c.Air_Time]).where(Flights.c.Air_Time >100)

result2 = conn.execute(query2)
for row2 in result2:
    print(row2)

('LAX', 'IAD', 267)
('LAX', 'IAD', 253)
('SFO', 'EWR', 312)
('JFK', 'SFO', 331)
('LAX', 'OGG', 309)
('OGG', 'LAX', 302)
('SFO', 'JFK', 288)
('LAX', 'EWR', 290)
('EWR', 'LAX', 321)
('LAX', 'EWR', 279)
('EWR', 'LAX', 320)
('LAX', 'EWR', 291)
('EWR', 'LAX', 326)
('EWR', 'LAX', 324)
('LAX', 'EWR', 266)
('SFO', 'EWR', 315)
('SFO', 'EWR', 292)
('SEA', 'LAX', 124)
('SFO', 'EWR', 278)
('EWR', 'SFO', 328)
('EWR', 'SFO', 325)
('EWR', 'SFO', 364)
('EWR', 'SFO', 360)
('SFO', 'DCA', 273)
('SFO', 'ORD', 222)
('ORD', 'SFO', 251)
('SFO', 'AUS', 176)
('AUS', 'SFO', 200)
('AUS', 'SFO', 195)
('SFO', 'AUS', 177)
('SFO', 'JFK', 346)
('ORD', 'LAX', 230)
('LAX', 'ORD', 215)
('LAX', 'ORD', 215)
('JFK', 'SFO', 331)
('ORD', 'LAX', 232)
('SFO', 'JFK', 334)
('JFK', 'LAS', 297)
('JFK', 'SFO', 333)
('LAS', 'JFK', 268)
('SFO', 'JFK', 354)
('JFK', 'SFO', 372)
('JFK', 'SFO', 347)
('DCA', 'SFO', 332)
('LAX', 'MCO', 262)
('MCO', 'LAX', 293)
('FLL', 'LAX', 319)
('LAX', 'FLL', 268)
('FLL', 'LAX', 303)
('SFO', 'FLL', 289)


In [8]:
#from sqlalchemy import select
#from sqlalchemy import func

#query = select([Flights.c.Origin, Flights.c.Destination, func.max(Flights.c.Air_Time)])
             
#result = conn.execute(query)
#for row in result:
#    print(row)

# Join
Join two tables into one that only contains entries with have air time greater than 100

In [9]:
q = select([Flights.c.Distance, Flights.c.Air_Time, Airports.c.City, Airports.c.State]).where(Flights.c.Origin==Airports.c.Code)

result3 = conn.execute(q)
for row in result3:
    print(row)

(2288, 267, 'Los Angeles, CA', 'CA')
(2288, 253, 'Los Angeles, CA', 'CA')
(447, 68, 'San Diego, CA', 'CA')
(2565, 312, 'San Francisco, CA', 'CA')
(2586, 331, 'New York, NY', 'NY')
(2485, 309, 'Los Angeles, CA', 'CA')
(2485, 302, 'Kahului, HI', 'HI')
(2586, 288, 'San Francisco, CA', 'CA')
(2454, 290, 'Los Angeles, CA', 'CA')
(2454, 321, 'Newark, NJ', 'NJ')
(2454, 279, 'Los Angeles, CA', 'CA')
(2454, 320, 'Newark, NJ', 'NJ')
(2454, 291, 'Los Angeles, CA', 'CA')
(2454, 326, 'Newark, NJ', 'NJ')
(2454, 324, 'Newark, NJ', 'NJ')
(2454, 266, 'Los Angeles, CA', 'CA')
(679, 98, 'San Francisco, CA', 'CA')
(679, 93, 'Seattle, WA', 'WA')
(2565, 315, 'San Francisco, CA', 'CA')
(2565, 292, 'San Francisco, CA', 'CA')
(954, 124, 'Seattle, WA', 'WA')
(2565, 278, 'San Francisco, CA', 'CA')
(2565, 328, 'Newark, NJ', 'NJ')
(2565, 325, 'Newark, NJ', 'NJ')
(337, 56, 'San Francisco, CA', 'CA')
(337, 0, 'Los Angeles, CA', 'CA')
(2565, 364, 'Newark, NJ', 'NJ')
(337, 58, 'San Francisco, CA', 'CA')
(2565, 360, 'N

In [10]:
# Using .join

meta = MetaData(engine, reflect=True)
Flights_t = meta.tables['Flights']
Airports_t = meta.tables['Airports']



join_obj = Flights_t.join(Airports_t,
                         Flights_t.c.Origin==Airports_t.c.Code)

sel_st = select(
   [Flights_t.c.Distance, Flights_t.c.Air_Time, Airports_t.c.City, Airports_t.c.State]).select_from(join_obj)

res = conn.execute(sel_st)

for _row in res:
    print(_row)

C:\Users\jwilli81\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  This is separate from the ipykernel package so we can avoid doing imports until


(2288, 267, 'Los Angeles, CA', 'CA')
(2288, 253, 'Los Angeles, CA', 'CA')
(447, 68, 'San Diego, CA', 'CA')
(2565, 312, 'San Francisco, CA', 'CA')
(2586, 331, 'New York, NY', 'NY')
(2485, 309, 'Los Angeles, CA', 'CA')
(2485, 302, 'Kahului, HI', 'HI')
(2586, 288, 'San Francisco, CA', 'CA')
(2454, 290, 'Los Angeles, CA', 'CA')
(2454, 321, 'Newark, NJ', 'NJ')
(2454, 279, 'Los Angeles, CA', 'CA')
(2454, 320, 'Newark, NJ', 'NJ')
(2454, 291, 'Los Angeles, CA', 'CA')
(2454, 326, 'Newark, NJ', 'NJ')
(2454, 324, 'Newark, NJ', 'NJ')
(2454, 266, 'Los Angeles, CA', 'CA')
(679, 98, 'San Francisco, CA', 'CA')
(679, 93, 'Seattle, WA', 'WA')
(2565, 315, 'San Francisco, CA', 'CA')
(2565, 292, 'San Francisco, CA', 'CA')
(954, 124, 'Seattle, WA', 'WA')
(2565, 278, 'San Francisco, CA', 'CA')
(2565, 328, 'Newark, NJ', 'NJ')
(2565, 325, 'Newark, NJ', 'NJ')
(337, 56, 'San Francisco, CA', 'CA')
(337, 0, 'Los Angeles, CA', 'CA')
(2565, 364, 'Newark, NJ', 'NJ')
(337, 58, 'San Francisco, CA', 'CA')
(2565, 360, 'N

(2288, 263, 'Los Angeles, CA', 'CA')
(1005, 138, 'Chicago, IL', 'IL')
(413, 107, 'Washington, DC', 'VA')
(1440, 167, 'Phoenix, AZ', 'AZ')
(1416, 218, 'New York, NY', 'NY')
(391, 68, 'Denver, CO', 'CO')
(1172, 169, 'Washington, DC', 'VA')
(1744, 198, 'Los Angeles, CA', 'CA')
(1744, 212, 'Los Angeles, CA', 'CA')
(447, 65, 'San Diego, CA', 'CA')
(1400, 205, 'Newark, NJ', 'NJ')
(678, 100, 'Chicago, IL', 'IL')
(2227, 300, 'Newark, NJ', 'NJ')
(1400, 160, 'Houston, TX', 'TX')
(1440, 193, 'Chicago, IL', 'IL')
(2007, 268, 'San Juan, PR', 'PR')
(680, 93, 'Denver, CO', 'CO')
(909, 113, 'Sacramento, CA', 'CA')
(867, 108, 'Chicago, IL', 'IL')
(1721, 195, 'Seattle, WA', 'WA')
(733, 99, 'Chicago, IL', 'IL')
(1167, 150, 'New Orleans, LA', 'LA')
(746, 98, 'Atlanta, GA', 'GA')
(836, 127, 'Denver, CO', 'CO')
(888, 121, 'Chicago, IL', 'IL')
(329, 50, 'San Francisco, CA', 'CA')
(1199, 135, 'Denver, CO', 'CO')
(224, 47, 'Raleigh/Durham, NC', 'NC')
(528, 66, 'Chicago, IL', 'IL')
(1739, 207, 'Portland, OR', '